# Data Crawling 
- Phần này dùng để chuẩn bị dữ liệu cho việc huấn luyện mô hình dự đoán giá điện thoại dựa vào một số đặc điểm của nó.
- Dữ liệu được thu thập từ các nhà bán lẻ điện thoại tại Việt Nam : Cellphones, Thegioididong, ... 
- Một số trường dữ liệu cần thiết cho việc dự đoán : </br>
    1. 


In [3]:
import pandas as pd 
import requests
import bs4
import numpy as np 
import time , os, json

In [6]:
class GMSarena_crawler : 
    """
    Khởi tạo nhận brand và id của brand 
    - Lưu đường dẫn html
    - Lưu đường dẫn tới file lưu 
    
    """
    def __init__(self, brand: str, brand_id: int): 
        self.df = pd.DataFrame(data = [])
        self.path = "https://www.gsmarena.com/" + brand + "-phones-" + str(brand_id) + ".php"
        self.save_path = ""
        self.feature = []


    """Dùng để crawl thông tin của một trang, trả về đối tương soup """    
    def crawl_page(self,  link): 
        
        time.sleep(30)

        try: 
            page = requests.get(link, timeout=5)
            
            soup = bs4.BeautifulSoup(page.text, "html.parser")

            return soup
        
        except ConnectionError as err : 
            print("Connection Error!")
            exit()

        except Exception as err : 
            print("Error!")
            exit()

    
    

    """Hàm này crawl các đường link của từng model trong trang gốc"""
    def crawl_model_link(self, soup):
        models = soup.find_all(attrs={"class" : 'makers'})
        href_val = []
        print(models)
        list_a = models[0].find_all('a')
        for a in list_a: 
            href_val.append(a['href'])
         
        return href_val
    

    
    """Hàm này crawl thông tin của một điện thoại"""
    def crawl_model(self, href_val):

        for i in href_val :
            soup = self.crawl_page('https://www.gsmarena.com/' + i)
            spec_list = soup.find_all(attrs = {"id" : 'specs-list'})
            specs = self.crawl_spec_table(spec_list[0])
            
        
    
    def test(self): 
        soup_page = self.crawl_page(self.path)
        href_val = self.crawl_model_link(soup_page)
        self.crawl_model(href_val)


    """Hàm này crawl giá của một model"""
    def crawl_price(self): 
        pass


    """ Hàm này crawl dữ liệu spec của 1 model """
    def crawl_spec_table(self, spec_list):
        specs = {}
        tables = spec_list.find_all('table')
        

        for table in tables : 
            ttl_a_tags = table.find_all('td', class_='ttl')
            nfo_a_tags = table.find_all('td', class_='nfo')
            pre_key = table.find('th').get_text()
            for i in range(min(len(ttl_a_tags), len(nfo_a_tags))):
            
                key = ttl_a_tags[i].find('a').get_text()
                val = nfo_a_tags[i].find('a').get_text()

                specs[pre_key + key] = val
        

        return specs        


    """ Hàm này save dữ liệu xuống file """
    def save(self): 
        pass 



    """Hàm này để test tính năng"""

# Test

In [7]:
test = GMSarena_crawler('samsung', 9)
tmp =  test.test()
tmp

[]


IndexError: list index out of range

In [8]:
page = requests.get('https://www.gsmarena.com/samsung-phones-9.php', timeout=5)
page

<Response [429]>

In [9]:
soup = bs4.BeautifulSoup(page.text, "html.parser")
soup

<html>
<head>
<title>Too Many Requests</title>
</head>
<body>
<h1>Too Many Requests</h1>
</body>
</html>

In [4]:
models = soup.find_all(attrs={"class" : 'makers'})
models[0]

<div class="makers">
<ul>
<li><a href="samsung_galaxy_c55-12945.php"><img src="https://fdn2.gsmarena.com/vv/bigpic/samsung-galaxy-c55.jpg" title="Samsung Galaxy C55 Android smartphone. Announced Apr 2024. Features 6.7″  display, Snapdragon 7 Gen 1 chipset, 5000 mAh battery, 256 GB storage, 12 GB RAM."/><strong><span>Galaxy C55</span></strong></a></li><li><a href="samsung_galaxy_m55-12896.php"><img src="https://fdn2.gsmarena.com/vv/bigpic/samsung-galaxy-m55.jpg" title="Samsung Galaxy M55 Android smartphone. Announced Mar 2024. Features 6.7″  display, Snapdragon 7 Gen 1 chipset, 5000 mAh battery, 256 GB storage, 12 GB RAM."/><strong><span>Galaxy M55</span></strong></a></li><li><a href="samsung_galaxy_tab_s6_lite_(2024)-12897.php"><img src="https://fdn2.gsmarena.com/vv/bigpic/samsung-galaxy-tab-s6-lite-2024.jpg" title="Samsung Galaxy Tab S6 Lite (2024) Android tablet. Announced Mar 2024. Features 10.4″  display, 7040 mAh battery, 128 GB storage, 4 GB RAM."/><strong><span>Galaxy Tab S6 Lit

In [5]:
models[0].find_all('a')[0].get('href')

'samsung_galaxy_c55-12945.php'